In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/3.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/4.csv").iloc[0:len(postulaciones2)//2], ignore_index=True)
print(len(postulaciones2))
postulaciones2.head()

9906801


,idaviso,idpostulante,sepostulo
0,1112257047,ow2xPaj,0
1,1111920714,LNKXqJB,0
2,1112346945,8MBPqYD,0
3,1112345547,ek4P2Wr,0
4,1112237522,NpqMD5,0


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16450211


In [9]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
14007757,1112363599,A3vvvlG,1,MASC,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,False,False,1.0
3800804,1112288714,X9kKr8Q,1,FEM,23.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
14303283,1112424849,pzd0Qpm,1,MASC,30.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
5288414,1112356607,wVO9PYV,0,FEM,27.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,False,False,NaN
14215964,1112457348,68JQbL,0,MASC,29.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
11183789,1112285937,Nzd10Nx,1,MASC,32.0,-,-,-,-,Graduado,...,True,True,False,True,False,False,False,False,False,NaN
12514375,1112305699,OqbYb4p,0,FEM,23.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
9423787,1112426771,OqdX1G6,0,MASC,40.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
2545497,1112299531,X95zJdx,0,MASC,19.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
10676019,1112438500,OqbWVrN,1,FEM,21.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,2.0


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
13625426,1112369464,0zrB35r,0,MASC,54.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
1506832,1112317170,N9EWVz,1,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
2315991,1112315655,5mq5RYZ,0,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.0
6567446,1112446506,NGJNbz,1,MASC,30.0,-,-,-,Graduado,Graduado,...,False,False,True,False,False,True,False,False,False,1.0
12926889,1111926163,5O3xMN,1,FEM,33.0,-,-,-,Graduado,Graduado,...,False,True,False,True,False,False,False,False,False,0.0
12633278,1112275075,X95eRXL,0,MASC,21.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
7081535,1112285920,lDb5NoG,1,FEM,55.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
4165170,1112344907,8M2Qmxz,1,FEM,20.0,-,-,-,-,Graduado,...,False,True,True,False,False,False,False,True,False,0.0
9529983,1112359332,EzELo84,1,FEM,44.0,-,-,-,-,Graduado,...,False,True,True,False,False,False,False,False,False,1.0
127426,1112323190,lDL1DWR,0,MASC,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0


In [11]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0

Train:  14560399 Test:  766337


In [12]:
gc.collect()

156

In [13]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=6000,max_depth=13,n_estimators=3000,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [14]:
score=c.score(x_test,y_test)*100
print(score)

76.19728657235655


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [15]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV10.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV10.pkl']

In [16]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
82035,29974,1112375007,akB6ZE,FEM,43.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
43862,24766,1112322603,1QP86qb,FEM,19.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,0.0
79726,53405,1112445074,e0OKo4,FEM,30.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,0.0
45682,1902,1111156211,8wReJz,NO_DECLARA,200.0,-,-,-,-,-,...,False,False,True,False,True,False,False,True,False,0.0
88284,44510,1112426713,5YdbX,FEM,42.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,False,False,0.0
32890,72760,1112463196,1lYZrb,FEM,34.0,-,-,-,-,-,...,False,False,False,False,False,False,False,True,False,0.0
65935,56958,1112450940,8XjKql,MASC,34.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
20521,83452,1112466383,rmdw9Bb,MASC,31.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,3.0
73526,34580,1112408912,ZDaL0R1,FEM,30.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,1.0
23621,68374,1112460662,8Mra1xM,FEM,22.0,-,-,En Curso,-,-,...,False,True,False,False,False,True,False,False,False,2.0


In [17]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.81140568, 0.18859432],
       [0.67176224, 0.32823776],
       [0.68360067, 0.31639933],
       ...,
       [0.91176283, 0.08823717],
       [0.70503585, 0.29496415],
       [0.97242383, 0.02757617]])

In [18]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [19]:
prediccion['sepostulo']=y_final2

In [20]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.188594
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.328238
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.316399
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.087395
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.264614


In [21]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.188594
1,1,0.328238
2,2,0.316399
3,3,0.087395
4,4,0.264614


In [22]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion24.csv",index=False)

In [23]:
prediccion.sepostulo.mean()

0.5341868423619432